In [3]:
# This program aims to generate Data Sequences
# A A Data Sequence consists of the following parts:
# 1. A PCR primer pair which can also be used as Chapter Number
# 2. A 4-nt base-3 converted sequence which denotes the Segment Number
# 3. Series of 7-nt Data Units which represents a 16-bit binary number. The first 6-nt refers to the Group Index, the last "Pointer Base" can be used to map the Reference Pool.
# 4. 12-nt base-3 converted error correction code which can correct up to 1 substitution error in the original Data Sequence.


import math
from array import *
import collections
from reedsolo import RSCodec

baseNumber = 0
baseNumber_complete = 0

# 2 error correction codes
rsc = RSCodec(2)

%store -r my_hexdata
%store -r Groups

bitsNumber = len(my_hexdata)*4

# Define Base Mapping Matrix
baseMappingMatrix = ['ACGT', 'CGTA', 'GTAC', 'TACG']

%store -r primerLibrary
%store -r segmentedChapteredData

def DNA_reverse_complement(DNA):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join(complement.get(base, base) for base in reversed(DNA))

def getPosition(String):
    position_list = []
    for i in range (0, 16):
        if String[i] == '1':
            position_list.append(i)
    return position_list

def converter(seq):
    converter = {'A': '00', 'C': '01', 'G': '10', 'T': '11'} 
    bases = list(seq) 
    bases = [converter[base] for base in bases] 
    return ''.join(bases)

def deconverter(seq):
    deconverter = {'00': 'A', '01': 'C', '10': 'G', '11': 'T'} 
    doubleBits = [seq[i:i+2] for i in range(0, len(seq), 2)]
    doubleBits = [deconverter[doubleBit] for doubleBit in doubleBits] 
    return ''.join(doubleBits)

def GCPercent(seq):
    counts = collections.Counter(seq)
    percent = (counts['C'] + counts['G'])/float(len(seq)) 
    return percent

def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))
    
def base3ToOligo(seq):
    base3ToOligo = {'0': 'G', '1': 'T', '2': 'A'}
    base3ToOligoConverted = []
    for i in range (0, len(seq)):
        base3ToOligoConverted.append(base3ToOligo[seq[i]])
        if base3ToOligo[seq[i]] == 'C':
            base3ToOligo = {'0': 'G', '1': 'T', '2': 'A'}
        elif base3ToOligo[seq[i]] == 'G':
            base3ToOligo = {'0': 'T', '1': 'A', '2': 'C'}
        elif base3ToOligo[seq[i]] == 'T':
            base3ToOligo = {'0': 'A', '1': 'C', '2': 'G'}
        elif base3ToOligo[seq[i]] == 'A':
            base3ToOligo = {'0': 'C', '1': 'G', '2': 'T'}
    return ''.join(base3ToOligoConverted)

def molmass(DNA):
    weight = 0
    single_mass = [313.2, 304.2, 289.2, 329.2]
    for i in range (0, len(DNA)):
        if DNA[i] == 'A':
            weight += single_mass[0]
        elif DNA[i] == 'T':
            weight += single_mass[1]
        elif DNA[i] == 'C':
            weight += single_mass[2]
        elif DNA[i] == 'G':
            weight += single_mass[3]
    return weight

def molconc(mass_conc, mol_weight):   # The unit of mass_conc is 'ng/uL', the unit of mol_weight is 'g/mol', the unit of the result mol_conc is 'nM'
    mol_conc = (mass_conc * 10**6) / (mol_weight * 10**9) *10**9
    return mol_conc
    
    
chapteredDataBinaryTotal = []    

scale = 16 ## equals to hexadecimal
num_of_bits = 8
mass = 0
GCTotal = 0
count = 0
chapteredDataBinary = []
array_full = [[None]*100 for _ in range(40)]
array_data = [[None]*100 for _ in range(40)]
array_data_payload = [[None]*100 for _ in range(40)]

# Preparation of segmented data whose length should be a multiple of 4 to facilitate RS correction
for i in range (0, len(segmentedChapteredData)):
    for k in range (0, len(segmentedChapteredData[i])):

        a = bin(int(segmentedChapteredData[i][k], scale))[2:].zfill(num_of_bits)
        if (len(a) % 4 != 0):
            a = '0' * (4-(len(a)%4)) + a
        
        chapteredDataBinary.append(a)

    chapteredDataBinaryTotal.append(chapteredDataBinary)
    chapteredDataBinary = []
    
# Assembling Data Sequences
for Chapter in range (0, len(chapteredDataBinaryTotal)):
    for Segment in range (0, len(chapteredDataBinaryTotal[Chapter])):
    
        originalData = chapteredDataBinaryTotal[Chapter][Segment]
        
        originalDataList = []
        dataPositions = []
        dataPositionsUnits = []
        orthogonalUnits = []

        arrayGap = 16
        
        arraySegmentNumber = int(len(originalData)/arrayGap)
        for j in range (0, arraySegmentNumber):
            originalDataSegment = originalData[arrayGap*j:arrayGap*(j+1)]
            originalDataList.append(originalDataSegment)
            dataPositionsUnits = getPosition(originalDataSegment)   
            dataPositions.append(dataPositionsUnits)

            dataPositionsUnits = []

        firstIndex = []
        dataSequence = []
        dataSequence_tuned = []
        for i in range (0, len(dataPositions)):
            for j0 in range (0, len(Groups)):
                for j1 in range (0, len(Groups[j0])):
                    if dataPositions[i] == Groups[j0][j1]:
                        firstIndex.append(j0)
                        ternaryNumber = ternary(j0)
                        ternaryNumber = '0'*(6-len(ternaryNumber)) + ternaryNumber
                        dataSequence.append(base3ToOligo(ternaryNumber))
                        dataSequence_tuned.append(base3ToOligo(ternaryNumber))
                        dataSequence.append(baseMappingMatrix[j1][j0%4])
                        dataSequence_tuned.append(baseMappingMatrix[j1][j0%4])   # Create dataSequence for RS correction
                        dataSequence_tuned.append(baseMappingMatrix[j1][j0%4])   # Supplement one base to make the length of the converted binary number exactly 16, thereby convenient for RS correction

        dataSequence = ''.join(dataSequence)
        array_data_payload[Chapter][Segment] = dataSequence
        dataSequence_tuned = ''.join(dataSequence_tuned)

        percent = GCPercent(dataSequence)

        ternarySegmentNumber = ternary(Segment)
        ternarySegmentNumber = '0'*(4-len(ternarySegmentNumber)) + ternarySegmentNumber

        ternarySegmentNumberSeq = base3ToOligo(ternarySegmentNumber)
        
        dataSequence = ternarySegmentNumberSeq + dataSequence
        dataSequence_tuned = ternarySegmentNumberSeq + dataSequence_tuned
        
        binaryConverted = converter(dataSequence_tuned)

        binaryList = [int(binaryConverted[i:i + 8], 2) for i in range(0, len(binaryConverted), 8)]
        
        byteArray = rsc.encode(binaryList)

        byte_to_ternary_oligo = []
        for byte in byteArray[-2:]:
            byte_to_ternary = ternary(byte)
            byte_to_ternary = '0'*(6-len(byte_to_ternary)) + byte_to_ternary
            byte_to_ternary_oligoUnit = base3ToOligo(byte_to_ternary)
            byte_to_ternary_oligo.append(byte_to_ternary_oligoUnit)
        
        byte_to_ternary_oligo = ''.join(byte_to_ternary_oligo)

        
        baseDeconverted = primerLibrary[Chapter] + dataSequence + byte_to_ternary_oligo + primerLibrary[len(primerLibrary)-Chapter-1]
        array_data[Chapter][Segment] = dataSequence + byte_to_ternary_oligo
        array_full[Chapter][Segment] = baseDeconverted
        
        print('This is Chapter '+ str(Chapter))
        print('This is Segment '+ str(Segment))
        count += 1

        print(baseDeconverted)
        # Calculating GC content of each strand
        GC = GCPercent(baseDeconverted)
        print('GC content of this strand: ' + str(GC))
        
        GCTotal += GCPercent(baseDeconverted)
        mass += molmass(baseDeconverted)
        percent = GCPercent(baseDeconverted)

        print("Oligo Length: " + str(len(baseDeconverted)))
        
        baseNumber += len(dataSequence)
        baseNumber_complete += len(baseDeconverted)
        

ref_base = 80*9*16        
ref_base_complete = 142*9*16
totalBaseNumber = baseNumber + ref_base
totalBaseNumber_complete = baseNumber_complete + ref_base_complete

print("Number of bits: " + str(bitsNumber))
file_size = bitsNumber/8/1000
print('File size: ' + str(file_size) + 'KB')
print('Number of Bases in the payload of Data Pool: ' + str(baseNumber))
print('Number of Bases in the payload of Reference Pool: ' + str(ref_base))
print('Number of total payload bases: ' + str(totalBaseNumber))
print('Number of Bases in complete Data Pool: ' + str(baseNumber_complete))
print('Number of Bases in complete Reference Pool: ' + str(ref_base_complete))
print('Number of total bases: ' + str(totalBaseNumber_complete))

# Calculation of coding potential
codingPotential_payload = bitsNumber/totalBaseNumber
print('Payload coding potential: ' + str(codingPotential_payload))
codingPotential_complete = bitsNumber/totalBaseNumber_complete
print('Overall coding potential: ' + str(codingPotential_complete))

# Calculating average GC content
GCAverage = GCTotal/count
print('Average GC content: ' + str(GCAverage))
# Calculation of concentration of the dissolved oligo pool
print("Total molecular weight: " + str(mass) + ' g/mol')
mass_concentration = 0.434
molecular_concentration = molconc(mass_concentration, mass)
print("Oligo pool concentration: " + str(molecular_concentration) + ' nM')

%store array_full
%store array_data
%store array_data_payload

print(count)


This is Chapter 0
This is Segment 0
ACTCCTCGAATGCTAGCGTTGGTACGTACGTAGTACGACGTACGCGGTACTATGTACTCAGTACTGCGTACACGGTACAGTGTACATAATGAGCAAGCACTAAGCACTAGCTCTGGTCATAACAGACACGTTCGATACCGCA
GC content of this strand: 0.4859154929577465
Oligo Length: 142
This is Chapter 0
This is Segment 1
ACTCCTCGAATGCTAGCGTTGGTAGGTAGTACGTAGTCGGTAGTGTGTAGACAGTAGAGCGTACATAGTAGATGGTAGCGTGTAGCTAGTAGCACGTATACGGTATAGTGTATATAGTATCGCGTATCTGGCTAGAGTGCTCACAGACACGTTCGATACCGCA
GC content of this strand: 0.4723926380368098
Oligo Length: 163
This is Chapter 0
This is Segment 2
ACTCCTCGAATGCTAGCGTTGGTATGTATCATGTATGTAGTATGACGTACAGTGTACATAGTATGCGGTCGTATGTATATAGTCGTCAGTCGTGCGTATCTGGTCGACGGTCGAGTGTCGATAAGCACTAGTCGTGGCGAGCACAGACACGTTCGATACCGCA
GC content of this strand: 0.48466257668711654
Oligo Length: 163
This is Chapter 0
This is Segment 3
ACTCCTCGAATGCTAGCGTTGGTCGGTAGTCGGTCGCGCGTCGCTGGTATCTGGTCGCATGTCTACAGTCTAGCGTCTATGGTCTCGTGTCTCTAGTCTCACGTCTGTGGTCTGATGTCTGCAAGCACTAGCGTCTGTGAGCACAGACACGTTCGATACCGCA
GC content of this strand: 0

This is Chapter 1
This is Segment 3
ACATCGCCATTCCAAGAGCATGTCGGCTGTCAGCTGTGCGCTCGCAGTATCTGGATCGCGGCTGACGGTAGCGTGACTAGTGTCTCACGTCTGTGGACACTAGACGAGCGACGCGTGTCTGTGGTGTCTGCTGCATACCAAGCTTCGGACTCCAAC
GC content of this strand: 0.5769230769230769
Oligo Length: 156
This is Chapter 1
This is Segment 4
ACATCGCCATTCCAAGAGCATGTCTGACGCACGTCTGATGCTGAGTGACGCGTGACTAGTGTAGCACGACATACGACATCGGCTCAGCGTCGAGTGACATACGTCTATGGCTGATAGACACACGCGCGACAGCACTAGACGATGCTCGCTACCAAGCTTCGGACTCCAAC
GC content of this strand: 0.5470588235294118
Oligo Length: 170
This is Chapter 1
This is Segment 5
ACATCGCCATTCCAAGAGCATGTCAGCTGTATGACGTCGGTAGATGGTGAGCAGTCATATGTCTATGGTCGTATGATCTATGACTGCGGAGAGACGCTGCGCGCTGTCAGCTGTGCGCTCGCAGTATCTGGCAGATGTCTCTTACCAAGCTTCGGACTCCAAC
GC content of this strand: 0.5276073619631901
Oligo Length: 163
This is Chapter 1
This is Segment 6
ACATCGCCATTCCAAGAGCATGTGTGCTGCTGGCTGCATGCACGTAATACATAGACGTGTGACAGATGCACGACGATCGACGAGACATGTCACGCGCACGCGGCGCTGCGACGAGCGTCTCGTGTCTCTAGTCACTGTCGCATACCAAGCTTCGGACTCCAAC
GC conte

This is Chapter 2
This is Segment 16
AGATTGCACGTGTGTCTGCCTGATCGACACGTGAGACATTAGACACGCAGATGGTCGCGCGCGATGTTAGAGTGTAGAGATATACGAGGACACTAGTACGACGTACGCGGTACTATAGCACTCAGCAGTGGTCAGCGCTCGTTCCACGAGCCATCTTCTCCAT
GC content of this strand: 0.5276073619631901
Oligo Length: 163
This is Chapter 2
This is Segment 17
AGATTGCACGTGTGTCTGCCTGATGGATCGTAGCGCTGCGTACTCAGTCGCATGTCTGTGGTCTACAAGCAGATGTGCACGGACTAGTGCTAGTAGAGACATGCGACTAGTAGAGCGCACGTAATAGTACGTATCTGGATGCAGAGACATCCACGAGCCATCTTCTCCAT
GC content of this strand: 0.5117647058823529
Oligo Length: 170
This is Chapter 2
This is Segment 18
AGATTGCACGTGTGTCTGCCTGCGTGCTAGTAGAGACATGCGACTAGTAGAGCTAGAGCAGTCGCGCGCGTCTGGTCTGCAGCGAGATGACGTCGGAGTATGGTCTATGGTCTCGTGACTGACGACGATGGATACGTGTACTAGCGATGTCCACGAGCCATCTTCTCCAT
GC content of this strand: 0.5470588235294118
Oligo Length: 170
This is Chapter 2
This is Segment 19
AGATTGCACGTGTGTCTGCCTGCGAGTGACTAGTATCTGGTCTCACTAGATACTAGATCGGAGTGATGTCGACGGCAGTGTGCGCTATGTGTCTGGTGTGACGCGACTAGTCACATGCATATAGACGCGTGACTGCGGCTATAGATATGTCCAC

This is Chapter 3
This is Segment 24
GTGGAACGCTTGACCAACTTGGCACTATGATAGTAGCACGACACGTGAGCTACGCTCTGTGTCTATGGTCTCGTGTCTCTAGTGCTATGTGTGACGCGACTAGTCACATGCATATATATGCGCGTAGTCGGTATCTGTACGTAGCACACCGAGGATGATCAACGCGAAGA
GC content of this strand: 0.5
Oligo Length: 170
This is Chapter 3
This is Segment 25
GTGGAACGCTTGACCAACTTGGCAGGCATCGCGTGCGTAGAGACATGTGCAGTGTACTATGTGTGCGGTGTGACGCGACTAGTCACATGCAGCGTGTCTGTGGCATAGTGCATCGCTATGCTGGTAGTCGGTCTGTGGATACGGCGTACCGAGGATGATCAACGCGAAGA
GC content of this strand: 0.5529411764705883
Oligo Length: 170
This is Chapter 3
This is Segment 26
GTGGAACGCTTGACCAACTTGGCATGTCGCATTAGCTGCGTCGTATGACAGCAGAGACATGACGATGGATACGTGTGACTAGTATCTGATACGTCATAGTAGGTGACTAGTCTATGAGCACTAGTGACTGATGCGCGAGGATGATCAACGCGAAGA
GC content of this strand: 0.48717948717948717
Oligo Length: 156
This is Chapter 3
This is Segment 27
GTGGAACGCTTGACCAACTTGTACGGACGTCGGAGAGCGGAGATATGACTAGTGTGCACGGTGTGACGAGATCATATCGCAGTGTGCGGTAGAGCTATGACGGTCACATTACTAGCGTACAGTGTGTCATGTATCTGGCTCACGCATAGCGAGGATGATCAACGCGAAGA
GC c

This is Chapter 4
This is Segment 20
AGATGGCATGGACACGGAGATGCGCAGCATAGATCGTCAATCGTGCATATAGAGTAGATGGTCGATATATACATGACTAGTGATGCGTGTGACTAAGCACTAAGCACTATACTATGCGTAGCGCAACTCTGTTCAAGGCTCA
GC content of this strand: 0.45774647887323944
Oligo Length: 142
This is Chapter 4
This is Segment 21
AGATGGCATGGACACGGAGATGCTAGTATCATGACACGTGCATGACGTCGCATGTGTGCGGATATACGATCGACGACACTAGTCTATGGTCGTATGACATACTCGTCGTTCGAGCGGAGTATGGTGTGCGGTATCTGGCATGCGTATGTCGCAACTCTGTTCAAGGCTCA
GC content of this strand: 0.5117647058823529
Oligo Length: 170
This is Chapter 4
This is Segment 22
AGATGGCATGGACACGGAGATGCTCGTACTCAGATATACGAGACGCGTGTCATGCACTGCGATAGATGTCTATGGTCTCGTGTCTCTAGTGCTATAGCAGTTAGCAGCAGACGATGAGCAGATATACTAAAGCAGTGGAGCTAGCAGCACGCAACTCTGTTCAAGGCTCA
GC content of this strand: 0.49411764705882355
Oligo Length: 170
This is Chapter 4
This is Segment 23
AGATGGCATGGACACGGAGATGCTGGTAGTCGGTCGCGCGCAGTACTCGCTCGGCGCTCAGCGCTGCGAGACATATCGAGTGTGTGCGTCGCTGTATCGATAGACGCGTAGCACTAGCTGAGGCAGAGCGCAACTCTGTTCAAGGCTCA
GC content of this stra

...

This is Chapter 35
This is Segment 34
AGTGCGAAGGAAGCCAACTAGTATCGTGCTATGTGTCGCGAGACTGGAGACATGATATGTGTGAGATATAGAGCGTATCATTACTCGTGTACATAGCGTCGCGACTAGTGTAGCACAGCACTAGCAGTGGCTGTCGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.4935897435897436
Oligo Length: 156
This is Chapter 35
This is Segment 35
AGTGCGAAGGAAGCCAACTAGTATGACGCGTAATGTCATACGCGACTGTCAGCAGCACACAGCACTAAGCACTAAGCACTAAGCACTAAGCACTAAGCACTAAGCACTAGACACAGTATCAGCAATAGACGCTCTCTCCAGT
GC content of this strand: 0.4788732394366197
Oligo Length: 142
This is Chapter 36
This is Segment 0
AGAGGAGACCACCAGCTCTTAGTACACGTGCGGTAGTCGTACGCTGTATGTATTATAGCGGACTGCGGACACTAGACGAGCGATATGTGTCTCACGTACAGTGTACATAATACAGTAGATCTAATCTATGAGCACTAGAGATCGTCGAGCACTTCACGTTCACACTGGAC
GC content of this strand: 0.4764705882352941
Oligo Length: 170
This is Chapter 36
This is Segment 1
AGAGGAGACCACCAGCTCTTAGTAGGTAGACAGTAGATGGTGTGACGCGACTAGACGCGTGCACACGGATAGATGTCACGCGTGTATAGTAGCTAGCTATATGCGACGTGTATCTGTGTCTCGGACTGCGGCTACAGTGCAGCACTTCACGTTCACACTGGAC
GC content of this stra

This is Chapter 36
This is Segment 49
AGAGGAGACCACCAGCTCTTATGAGGTGACTAGTATCTGAGTCGCGGACTCGCGTCGCTGGTAGCTAGTCTATGGCTCTGTGATCGCGGTGCGTAGTGTGACGAGATCATCGTATGGAGATCAGAGTCATACGAGCACTTCACGTTCACACTGGAC
GC content of this strand: 0.532051282051282
Oligo Length: 156
This is Chapter 36
This is Segment 50
AGAGGAGACCACCAGCTCTTATGATTAGCTGCGAGACTGGTAGCGTGACGATGTCGCTCGGAGTATGGCTACTGGTAGCACGATATGTGTGAGATGAGTACAGTCTCTAGTAGCGTACGATCACTTCACGTTCACACTGGAC
GC content of this strand: 0.5070422535211268
Oligo Length: 142
This is Chapter 36
This is Segment 51
AGAGGAGACCACCAGCTCTTATGCGTGAGTGTGATCGCGGTGTAGTGTCTGATGTAGATGGTAGCGTATGTGTAGTAGTCGGCTAGACGTAGTCGGAGTACAGTCTCTAAGCGTCGGTCGCATACGTGTAAGCACTAGACACAGACGACCACTTCACGTTCACACTGGAC
GC content of this strand: 0.5176470588235295
Oligo Length: 170
This is Chapter 36
This is Segment 52
AGAGGAGACCACCAGCTCTTATGCTGTGCATAGTATCTGGACTGTAGTCTGCAGACACACTAGCATAGACGTCGGTCTGTGGTGCTATGTAGATGAGCACTAAGCACTAGTGATCGACAGTCACTTCACGTTCACACTGGAC
GC content of this strand: 0.49295774647887

This is Chapter 37
This is Segment 19
ACTTGGCATGTCCTACATCGTGCGAGTGACTAGTATCTGGTGTGTAGTCGAGTGCTGTGCGTCTCGTGTCTCTAGTGCTATGTAGATGGTAGCTAGCTATCAGCGTCGCGTCTGTGGTGCTATAGCACTATACTAGGTCATAGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.49693251533742333
Oligo Length: 163
This is Chapter 37
This is Segment 20
ACTTGGCATGTCCTACATCGTGCGCGTGACTAGCAGCGTGTATCTGGATGCGTGTGACTAATACATAATGACTCATGAGCAGCGATACGCACATAGTCGCATGACACTAGCTGAGTGCTCAGCGTCACGCGACTAGTGAGCAGGCGACTGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.5235294117647059
Oligo Length: 170
This is Chapter 37
This is Segment 21
ACTTGGCATGTCCTACATCGTGCTAGATGCGTGTGACTAGTAGATGGACATACGCTAGTAGTCAGCGGTGCAGTGTAGTACGATAGATGTGCTCAGTAGACAGACGCGTGACTGTAAGCAGATGTATAGGTCAGAGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.48717948717948717
Oligo Length: 156
This is Chapter 37
This is Segment 22
ACTTGGCATGTCCTACATCGTGCTCGTCTGATGTCTGCAGTAGATGGTAGCTAGTCGCATTAGCGCGGTGATACGTGCTCAGCACACGGAGTATGGAGCATGGCATCTGGACGAGCGATAGATAGCACTAGTGTCTGCACGTGTCGAACGTGGACAGATTGG

This is Chapter 37
This is Segment 57
ACTTGGCATGTCCTACATCGTACTAGACACTAGACGCTAGAGAGACGACGAGCGATAGATGTAGATGTCACGATTCGATATGTGCGTAGTGAGCAGCGACGTGTGAGATGACGATGGATACGTGCGCTCGAGTGCGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.5128205128205128
Oligo Length: 156
This is Chapter 37
This is Segment 58
ACTTGGCATGTCCTACATCGTACTCTACATCAACGTAGTTCGATATGTCTATGGATATGTGCGCGTAGACACTAGTCACTGGACATGTGAGTATGGACTAGTGTCGCTGGTCGCATGTCATCAGCTATCGTCAGTGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.47435897435897434
Oligo Length: 156
This is Chapter 37
This is Segment 59
ACTTGGCATGTCCTACATCGTACTGTACATGCGTAGCGTGAGACGCGACGTCGGTACGCGGTACTATGTATAGTGCAGATGGTACAGTGCGCGTAGTAGTCGGTCGCGCGAGTCAGTGCAGGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.5492957746478874
Oligo Length: 142
This is Chapter 37
This is Segment 60
ACTTGGCATGTCCTACATCGTACACGCAGATGGCGACGTTAGTATAGTCTCGTGACACACGTACGACGACTACGGTACTATGCACGACGTACGCGAGCACACAGCACTAGAGAGTGTAGTAGTCGAACGTGGACAGATTGGT
GC content of this strand: 0.5140845070422535
Oligo Len

This is Chapter 39
This is Segment 23
ACCGTAGATCCTCGACAAGCTGCTGGAGAGACGCTGAGTGACTCGCGCACGACTATGTCAGCGTCGCGACTAGTGCGATGTGACAGATTGTGCATGCGCTGCATGAGCAGTAGTGTGTCACGCTATATATAGCACTAGTAGTAGTGATGAGGATGCCGTTCGTAGTGCTA
GC content of this strand: 0.5235294117647059
Oligo Length: 170
This is Chapter 39
This is Segment 24
ACCGTAGATCCTCGACAAGCTGCACATGAGCCATACGCTAGCACAGACTGCGCGTGTGCGTCGACATGTCGATAGACGACAGTGCAGTGTGCGTAGTCGCGCATACTAAAGCAGTGGTAGATGTAGACAGGATGCCGTTCGTAGTGCTA
GC content of this strand: 0.5369127516778524
Oligo Length: 149
This is Chapter 39
This is Segment 25
ACCGTAGATCCTCGACAAGCTGCAGTAGCTCAGACTACGATACTAAATCTGTGGTGTGACGAGATCAGTATCATGTACTCAGTGAGATGACTAGTGTAGTCGAGCACTAGTGATCGAGTGCAGGATGCCGTTCGTAGTGCTA
GC content of this strand: 0.4788732394366197
Oligo Length: 142
This is Chapter 39
This is Segment 26
ACCGTAGATCCTCGACAAGCTGCATGACGTCGGCATGCGGTATAGTGAGAGACGTCGCTGGTGTGACGTATACGGACTGACGTCATGCTCTGAGCGTCTGTGGACGTCGGACACGTGACACTAGTGTAGTAGCACTAGCTCAGGCAGCGAGGATGCCGTTCGTAGTGCTA
GC content of this st